# 필요 라이브러리 다운및 import

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 7.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595728 sha256=2b3e085b40dcba31703607aade2060f48624daec9278596d019b92c6da627913
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yadfbbtu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yadfbbtu
     |████████████████████████████████| 129 kB 7.5 MB/s 
     |████████████████████████████████| 54.7 MB 39 kB/s 
     |████████████████████████████████| 4.5 MB 75.8 MB/s 
     |████████████████████████████████| 1.2 MB 76.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x3a788000 @  0x7f56b5d6c615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 1

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 전처리

In [8]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/kmong/emotion_detect/data/text_data.csv',encoding='cp949')

In [9]:
len(df)

34453

In [10]:
df['emotion'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [11]:
df['emotion'].value_counts()

4    8325
0    6872
6    4950
3    4057
1    3958
2    3367
5    2924
Name: emotion, dtype: int64

In [12]:
index = df[df['emotion']==6].index

In [13]:
print(index)

Int64Index([29503, 29504, 29505, 29506, 29507, 29508, 29509, 29510, 29511,
            29512,
            ...
            34443, 34444, 34445, 34446, 34447, 34448, 34449, 34450, 34451,
            34452],
           dtype='int64', length=4950)


In [14]:
df = df.drop(index)

In [15]:
df['emotion'].value_counts()

4    8325
0    6872
3    4057
1    3958
2    3367
5    2924
Name: emotion, dtype: int64

In [16]:
df.sample(n=10)

,dialog,emotion
15474,다음 달이 출산일인데 담당 선생님이 나는 아기 잘 낳을 수 있을 거라고 하셨어.,3
12422,아빠가 가끔 어려운 이야기를 하실 때 이해 안 될 때가 있어서 혼란스러워.,2
20108,남자친구랑 연애하고 있는데도 외로운 느낌이 들어.,4
28059,어제 너무 부끄러운 일이 있었어.,5
29158,"친구들이 나한테 청첩장을 달라고 하는데, 난 친구들에게 내 결혼식을 알리고 싶지 않거든.",5
21877,선생님께 도움을 요청했더니 자신을 믿으라고 말씀하셨어. 품에 안겨서 펑펑 울었어.,4
3422,시험 보는데 갑자기 배가 아파 와서 짜증나.,0
3024,부모님이 나랑 언니랑 차별해. 나 너무 힘들고 짜증이 나.,0
17821,"차, 착각하지 마! 내년 내내 오빠랑 같이 있는단 얘긴 같은 고등학교에 다니게 된다...",3
20314,내 진로에 대해 생각해보고 싶은데 자꾸 성적이 앞길을 막아,4


In [17]:
data_list = []
for q,label in zip(df['dialog'], df['emotion']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

  

In [18]:
print(data_list[0])
print(data_list[1000])

[' 그건 그 여자 사정이지!', '0']
['나는  정말 이렇게 먹고는 못살아', '0']


In [19]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size = 0.25, random_state =0)

In [20]:
print(len(dataset_train))
print(len(dataset_test))

22127
7376


# BERT DATASET

In [21]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx])for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i],))

  def __len__(self):
    return(len(self.labels))

parameter

In [22]:
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


토큰화 과정

In [23]:
tokenizer = get_tokenizer()

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [24]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [25]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [26]:
data_train[2]

(array([   2,  517, 6248, 2829, 6844,  633,  806, 3466,  633,  517,    5,
         517,    5,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# BERT MODEL

In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [29]:
model = BERTClassifier(bertmodel,  dr_rate=0.5)

In [30]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [31]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [32]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [33]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [34]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# Training & save model

In [35]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        valid_length= valid_length
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()
        label = label.long()
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  1%|          | 1/173 [00:31<1:30:56, 31.73s/it]

epoch 1 batch id 1 loss 1.8208636045455933 train acc 0.1875


100%|██████████| 173/173 [1:25:50<00:00, 29.77s/it]

epoch 1 train acc 0.4967013617663907



100%|██████████| 58/58 [09:47<00:00, 10.12s/it]

...

epoch 5 batch id 1 loss 0.4355171024799347 train acc 0.8671875


100%|██████████| 173/173 [1:24:50<00:00, 29.43s/it]

epoch 5 train acc 0.8407109240743634



100%|██████████| 58/58 [09:44<00:00, 10.08s/it]

epoch 5 test acc 0.701697198275862


In [36]:
torch.save(model.state_dict(), '/content/drive/MyDrive/kmong/emotion_detect/model_1.h5')

# Load Model & Predict

In [43]:
model = BERTClassifier(bertmodel,  dr_rate=0.5)
model.load_state_dict(torch.load('/content/drive/MyDrive/kmong/emotion_detect/model_1.h5'))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [56]:
def predict(predict_sentence):
  
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long()
    segment_ids = segment_ids.long()

    valid_length = valid_length
    label = label.long()

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]

    for i in out:
      logits = i
      logits = logits.detach().numpy()

      if np.argmax(logits) == 0:
        test_eval.append("분노가")
      elif np.argmax(logits) == 1:
        test_eval.append("놀람이")
      elif np.argmax(logits) == 2:
        test_eval.append("불안이")
      elif np.argmax(logits) == 3:
        test_eval.append("행복이")
      elif np.argmax(logits) == 4:
        test_eval.append("슬픔이")
      elif np.argmax(logits) == 5:
        test_eval.append("당황이")
   

    print(">> 입력하신 내용에서" + test_eval[0] + " 느껴집니다.")
      

In [73]:
predict("정말 당황스럽다")

>> 입력하신 내용에서당황이 느껴집니다.
